DEPENDENCIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_validate
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

TRAIN DATA

In [ ]:
train_positive_1 = np.fromfile('exp1-train-400pos.bin', dtype=np.float32).reshape([400, 5, 768])
train_positive_2 = np.fromfile('exp2-train-400pos.bin', dtype=np.float32).reshape([400, 5, 768])
train_positive = np.concatenate([train_positive_1, train_positive_2])

train_negative_1 = np.fromfile('exp1-train-2000neg.bin', dtype=np.float32).reshape([2000, 5, 768])
train_negative_2 = np.fromfile('exp2-train-2000neg.bin', dtype=np.float32).reshape([2000, 5, 768])
train_negative = np.concatenate([train_negative_1, train_negative_2])

train_data = np.concatenate([train_positive, train_negative])

TEST DATA

In [ ]:
test_data = np.fromfile('exp12-test-1200.bin', dtype=np.float32).reshape([1200, 5, 768])

LABELS

In [ ]:
train_negative_labels = [len(i) * [0] for i in train_negative]
train_positive_labels = [len(i) * [1] for i in train_positive]

labels = np.concatenate([train_positive_labels, train_negative_labels])

GROUPS

In [ ]:
group = [[i] * len(j) for i, j in enumerate(train_data)]
groups = np.hstack(group)

SHORTCUTS

In [ ]:
#X_train = train_data
X_train = np.vstack(train_data)
#X_train.reshape(2400, 5, 768)
#Y_train = labels
Y_train = np.hstack(labels)
#Y_train.reshape(2400, 5)
X_test = np.vstack(test_data)
#X_test.reshape(1200, 5, 768)
#X_test = np.vstack(test_data)
#print(X_train.shape, Y_train.shape, X_test.shape)
#print(X_train[0])

STANDARD SCALER

In [ ]:
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

MODEL TRAINING - Logistic Regression

In [ ]:
model = LogisticRegression()
gkf = GroupKFold(n_splits = 5)
param_grid = {'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', model)])
gscv = GridSearchCV(pipe, param_grid, cv = gkf, n_jobs = 32)
gscv.fit(X_train, Y_train, groups = groups)
#model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             n_jobs=32,
             param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5,
                                           8, 10, 12, 15]})

MODEL TRAINING - SVM

In [ ]:
#model_2 = svm.SVC(kernel = 'linear')
#gkf = GroupKFold(n_splits = 5)
#param_grid = {'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}
#pipe = Pipeline([('scaler', StandardScaler()), ('classifier', model_2)])
#gscv = GridSearchCV(pipe, param_grid, cv = gkf, n_jobs = 1)
#gscv.fit(X_train, Y_train, groups = groups)
#model_2.fit(X_train, Y_train)

MODEL TRAINING - Decision Tree

In [ ]:
tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
tree.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

MODEL TRAINING - Random Forest Classifier

In [ ]:
#MODEL TRAINING FINAL

forest_FINAL = RandomForestRegressor(random_state = 0, max_features = 'auto', n_estimators = 500, min_samples_leaf=50)
forest_FINAL.fit(X_train, Y_train)

KeyboardInterrupt: ignored

MODEL EVALUATION - TRAINING DATA 

In [ ]:
#X_train_prediction = gscv.predict(X_train)
#training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
#print('Logistic Regression Accuracy: ', gscv.score(X_train, Y_train))
#print('Decision Tree Accuracy: ', tree.score(X_train, Y_train))
print('Random Forest Classifier Accuracy: ', forest_FINAL.score(X_train, Y_train))

Random Forest Classifier Accuracy:  0.5202392944075325


PREDICTION - TEST DATA

In [ ]:
Predictions = forest_FINAL.predict(X_test)

Predictions_Final = np.empty(1200, dtype = int)
i=0
l=0
while i<len(Predictions):
    j=i
    k=j+5
    while j<k:
        ok=0
        if(Predictions[j]==1):
          ok=1
          j=k
        j+=1
    if ok==0:
        Predictions_Final[l]=0
    else:
      Predictions_Final[l]=1
    l+=1
    i+=5
i=0 

CREATING A SUBMISSION FILE

In [ ]:
Id = np.array(range(0, 1200))
output = pd.DataFrame({'Outcome' : Predictions_Final, 'Id' : Id.tolist()})

In [ ]:
output.to_csv('sampleSubmission_randomforest1.csv', index=False)

In [ ]:
#INITIAL MODEL TRAINING (RANDOM FOREST CLASSIFIER - GRID SEARCH)

forest = RandomForestClassifier(random_state = 0)
#forest_GKF = GroupKFold(n_splits = 5)
#forest_PARAS = {'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}
#forest_GSCV = GridSearchCV(forest, forest_PARAS, cv = forest_GKF, n_jobs = 1)
#forest.fit(X_train, Y_train)
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
forest_GSCV = GridSearchCV(estimator=forest, param_grid=param_grid, cv = 5)
#forest_GSCV.fit(X_train, Y_train)
forest_GSCV.fit(X_train, Y_train, groups = groups)

NameError: ignored